In [1]:
import mysql.connector as mysq

mydb = mysq.connect(
host="localhost",
user="root",
password="password",
database="dataBucket"
)
mycursor = mydb.cursor()

-       Log        [ dateTime(DateTime) , userName(String), rewardClaimed(Boolean), billTotal(Int) ]
-       Rewards    [ dateTime_allocated(DateTime), dateTime_claimed(DateTime), userName(String), rewardToken(UNIQUE String), category(Int), status(String)]
-       Products   [ productCode(UNIQUE String),productName(String), productPrice(Int), UnitsSold(Int), UnitsSold_Reward(Int)]

In [2]:
import datetime
dt = datetime.datetime.now()

def prod_setup(productName, productCode, productPrice, UnitsSold, UnitsSold_Reward):
    mycursor.execute("INSERT INTO products (productCode, productName, productPrice, UnitsSold, UnitsSold_Reward) VALUES (%s,%s,%s,%s,%s);",(productCode, productName, productPrice, UnitsSold, UnitsSold_Reward))
    mydb.commit()
    #print("Inserted Into Products:")

def reward_setup(userName:str, rewardAllocated:str, category:int):
    mycursor.execute("INSERT into rewards (dateTime_allocated, dateTime_claimed, userName, rewardToken, category, status) VALUES (%s,%s,%s,%s,%s,%s);",(dt,None,userName, rewardAllocated, category,"ACTIVE"))
    mydb.commit()
    #print("Inserted Into rewards:")

def log_setup(userName, rewardClaimed, bill):
    mycursor.execute("INSERT into logs (dateTime, userName, rewardClaimed, billTotal) VALUES (%s,%s,%s,%s);",(dt, userName, rewardClaimed, bill ))
    mydb.commit()
    #print("Inserted Into log:")

# Products

In [3]:
mycursor.execute("TRUNCATE TABLE products;")
prod_setup("Laptop","LP",250,0,0)
prod_setup("CPU","CP",450,0,0)
prod_setup("Monitor","MN",150,0,0)
prod_setup("Cables","CA",50,0,0)
prod_setup("Keyboard","KB",100,0,0)
prod_setup("Mouse","MO",75,0,0)
prod_setup("RAM","RM",300,0,0)
prod_setup("GPU","GP",350,0,0)

mycursor.execute("SELECT * from products;")
myresult = mycursor.fetchall()
myresult

[('LP', 'Laptop', 250, 0, 0),
 ('CP', 'CPU', 450, 0, 0),
 ('MN', 'Monitor', 150, 0, 0),
 ('CA', 'Cables', 50, 0, 0),
 ('KB', 'Keyboard', 100, 0, 0),
 ('MO', 'Mouse', 75, 0, 0),
 ('RM', 'RAM', 300, 0, 0),
 ('GP', 'GPU', 350, 0, 0)]

# Reward

In [4]:
mycursor.execute("TRUNCATE TABLE rewards;")
reward_setup("pooshpal","LP%20",2)
reward_setup("pooshpal","CP%30",3)
reward_setup("tom","MN%10",1)
reward_setup("tom","CP%30",3)

mycursor.execute("SELECT * from rewards;")
myresult = mycursor.fetchall()
myresult

[(datetime.datetime(2022, 11, 8, 14, 23, 26),
  None,
  'pooshpal',
  'LP%20',
  2,
  'ACTIVE'),
 (datetime.datetime(2022, 11, 8, 14, 23, 26),
  None,
  'pooshpal',
  'CP%30',
  3,
  'ACTIVE'),
 (datetime.datetime(2022, 11, 8, 14, 23, 26),
  None,
  'tom',
  'MN%10',
  1,
  'ACTIVE'),
 (datetime.datetime(2022, 11, 8, 14, 23, 26),
  None,
  'tom',
  'CP%30',
  3,
  'ACTIVE')]

# Log

In [5]:
mycursor.execute("TRUNCATE TABLE logs;")
log_setup("pooshpal","True",400)
log_setup("pooshpal","False",100)
log_setup("tom","True",500)
log_setup("tom","False",400)

mycursor.execute("SELECT * from logs;")
myresult = mycursor.fetchall()
myresult

[(datetime.datetime(2022, 11, 8, 14, 23, 26), 'pooshpal', 'True', 400),
 (datetime.datetime(2022, 11, 8, 14, 23, 26), 'pooshpal', 'False', 100),
 (datetime.datetime(2022, 11, 8, 14, 23, 26), 'tom', 'True', 500),
 (datetime.datetime(2022, 11, 8, 14, 23, 26), 'tom', 'False', 400)]

In [6]:
userName = "pooshpal"
mycursor.execute("SELECT * from logs where userName = %s;",(userName,))
myresult = mycursor.fetchall()
myresult

[(datetime.datetime(2022, 11, 8, 13, 21, 53), 'pooshpal', 'True', 400),
 (datetime.datetime(2022, 11, 8, 13, 21, 53), 'pooshpal', 'False', 100)]

# tests

In [1]:
from Database import _database
db = _database()

display(db.getPast(userName="pooshpal"))
display(db.getProductName(code="LP"))

('LOYAL', 2, 1)

'Laptop'

In [3]:

rewardsToken = list(a[0] for a in db.getRewards(userName="pooshpal"))
rewardsToken

['LP%20', 'CP%30']

In [7]:
from Helper import helper
help = helper(userName="pooshpal")

token , eng_token = help.getUserRewards()
display(token)
display(eng_token)

['LP%20', 'CP%30']

['LP%20 - Laptop @ 20 %', 'CP%30 - CPU @ 30 %']

In [2]:
db.getProductInfo()

[('Laptop', 250),
 ('CPU', 450),
 ('Monitor', 150),
 ('Cables', 50),
 ('Keyboard', 100),
 ('Mouse', 75),
 ('RAM', 300),
 ('GPU', 350)]

In [2]:
help.getProductInfo()

(['Laptop', 'CPU', 'Monitor', 'Cables', 'Keyboard', 'Mouse', 'RAM', 'GPU'],
 [250, 450, 150, 50, 100, 75, 300, 350])

In [3]:
db.productNeedsHelp()

'LP'

In [4]:
help.allocateReward(bill=560)

'LP%30'

In [7]:
userName="pooshpal"
token="LP%20"
mycursor.execute("UPDATE rewards set status = 'CLAIMED',dateTime_claimed = %s WHERE userName = %s AND rewardToken = %s;",(dt,userName,token))
mydb.commit()
mycursor.execute("SELECT * from rewards;")
myresult = mycursor.fetchall()
myresult

[(datetime.datetime(2022, 11, 8, 13, 28, 11),
  datetime.datetime(2022, 11, 8, 13, 28, 11),
  'pooshpal',
  'LP%20',
  2,
  'CLAIMED'),
 (datetime.datetime(2022, 11, 8, 13, 28, 11),
  None,
  'pooshpal',
  'CP%30',
  3,
  'ACTIVE'),
 (datetime.datetime(2022, 11, 8, 13, 28, 11),
  None,
  'tom',
  'MN%10',
  1,
  'ACTIVE'),
 (datetime.datetime(2022, 11, 8, 13, 28, 11),
  None,
  'tom',
  'CP%30',
  3,
  'ACTIVE')]

In [12]:
display(help.getUserRewards())
help.rewardClaimed(token="LP%20")
display(help.getUserRewards())

(['LP%20', 'CP%30'], ['LP%20 - Laptop @ 20 %', 'CP%30 - CPU @ 30 %'])

(['LP%20', 'CP%30'], ['LP%20 - Laptop @ 20 %', 'CP%30 - CPU @ 30 %'])

In [17]:
from Helper import helper
help = helper("pooshpal")

help.updateProduct(purcahsedProducts={"Laptop":4},rewardClaimed="LP%20")
mydb.commit()
mycursor.execute("SELECT * from products;")
myresult = mycursor.fetchall()
myresult

[('LP', 'Laptop', 250, 48, 36),
 ('CP', 'CPU', 450, 0, 0),
 ('MN', 'Monitor', 150, 0, 0),
 ('CA', 'Cables', 50, 0, 0),
 ('KB', 'Keyboard', 100, 0, 0),
 ('MO', 'Mouse', 75, 0, 0),
 ('RM', 'RAM', 300, 0, 0),
 ('GP', 'GPU', 350, 0, 0)]